In [19]:
%load_ext autoreload
%autoreload 2

import ase.io
import numpy as np

import rascaline
import equistore
from equistore import Labels, TensorBlock, TensorMap
import chemiscope

# from rascaline.utils import clebsch_gordan
import clebsch_gordan
import spherical

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
frames = ase.io.read("combined_magres_spherical.xyz", ":")

# Define hyperparameters for generating the rascaline SphericalExpansion
rascal_hypers = {
    "cutoff": 3.0,  # Angstrom
    "max_radial": 6,  # Exclusive
    "max_angular": 5,  # Inclusive
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "center_atom_weight": 1.0,
}

# Define target lambda channels
lambdas = np.array([0, 1, 2])

In [12]:
chemiscope.show([frames[0], frame_inv], mode="structure")

/opt/miniforge3/envs/dev/lib/python3.10/site-packages/chemiscope/input.py:195: UserWarning: chemiscope behavior changed to no longer include properties from the structure objects. Use `chemiscope.extract_properties` to also visualize these properties ([efg_L2, efg, efg_L0])
  warnings.warn(


StructureWidget(value='{"meta": {"name": " "}, "structures": [{"size": 42, "names": ["Li", "Li", "Li", "Li", "…

# Equivariance Test - SO(3)

In [21]:
# Pick a test frame
frame = frames[0]

# Randomly rigidly rotate the frame
frame_rot, (a, b, c) = spherical.rotate_ase_frame(frame)
print("Random rotation angles (rad):", a, b, c)
assert not np.all(frame.positions == frame_rot.positions)

# Generate lambda-SOAP for both frames
lsoap = clebsch_gordan.n_body_iteration_single_center(
    [frame],
    rascal_hypers,
    nu_target=2,
    lambdas=lambdas,
)

lsoap_rot = clebsch_gordan.n_body_iteration_single_center(
    [frame_rot],
    rascal_hypers,
    nu_target=2,
    lambdas=lambdas,
)

# Build a Wigner-D Matrix from the random rotation angles
wig = spherical.WignerDReal(rascal_hypers["max_angular"], a, b, c)

# Rotate the lambda-SOAP descriptor of the unrotated frame
lsoap_unrot_rot = wig.rotate_tensormap(lsoap)

# Check for equivariance!
assert equistore.equal_metadata(lsoap_unrot_rot, lsoap_rot)
assert equistore.allclose(lsoap_unrot_rot, lsoap_rot)
print("SO(3) EQUIVARIANT!")

Random rotation angles (rad): 2.20723878026074 2.7663510356084258 2.130637286978451
SO(3)EQUIVARIANT!


# Equivariance Test - O(3)

In [48]:
# Pick a test frame
frame = frames[0]

# Invert the positions
frame_inv = frames[0].copy()
frame_inv.positions = -1 * frame_inv.positions
assert not np.all(frame.positions == frame_inv.positions)

# Generate lambda-SOAP for both frames
lsoap = clebsch_gordan.n_body_iteration_single_center(
    [frame],
    rascal_hypers,
    nu_target=2,
    lambdas=lambdas,
)

lsoap_inv = clebsch_gordan.n_body_iteration_single_center(
    [frame_inv],
    rascal_hypers,
    nu_target=2,
    lambdas=lambdas,
)

# Invert the TensorMaps
lsoap_uninv_inv = spherical.invert_tensormap(lsoap)

# Check for equivariance!
assert equistore.equal_metadata(lsoap_uninv_inv, lsoap_inv)
for key in lsoap_inv.keys:
    close = equistore.allclose_block(lsoap_uninv_inv[key], lsoap_inv[key])
    print(key, close)
    # if not close:
    #     break
# equistore.allclose_raise(lsoap_uninv_inv, lsoap_inv)
# print("INVERSION EQUIVARIANT!")

LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=0, species_center=3) True
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=1, species_center=3) False
LabelsEntry(order_nu=2, inversion_sigma=-1, spherical_harmonics_l=1, species_center=3) True
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=2, species_center=3) True
LabelsEntry(order_nu=2, inversion_sigma=-1, spherical_harmonics_l=2, species_center=3) False
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=0, species_center=8) True
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=1, species_center=8) False
LabelsEntry(order_nu=2, inversion_sigma=-1, spherical_harmonics_l=1, species_center=8) True
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=2, species_center=8) True
LabelsEntry(order_nu=2, inversion_sigma=-1, spherical_harmonics_l=2, species_center=8) False
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=0, species_center

In [49]:
lsoap_uninv_inv.block(order_nu=2, inversion_sigma=-1, spherical_harmonics_l=2, species_center=22).values[0, ..., 0]

array([0., 0., 0., 0., 0.])

In [51]:
diff = equistore.subtract(lsoap_uninv_inv, lsoap_inv)
diff.block(order_nu=2, inversion_sigma=-1, spherical_harmonics_l=2, species_center=22).values

array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          1.74112929e-08, -2.46996381e-08, -4.00459763e-08],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -5.43871155e-10,  7.70527867e-10,  1.25186725e-09],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -2.09929680e-09,  2.97451559e-09,  4.83177337e-09],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -4.21231684e-08,  5.97620466e-08,  9.68773874e-08],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          1.05008555e-09, -1.48888188e-09, -2.41593216e-09]],

       [[ 5.77641685e-09, -1.10520104e-08,  3.05784221e-08, ...,
          3.31438002e-06, -4.69593733e-06, -7.62865268e-06],
        [ 3.52113221e-11, -6.64820657e-11,  1.88645904e-10, ...,
         -1.00210410e-07,  1.41982231e-07,  2.30651638e-07],
        [ 8.62442322e-11, -1.62836678e-10,  4.62056526e-10, ...,
          1.00771627e-08, -1.42656815e-08, -2.32029

In [ ]:
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=0, species_center=3) True
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=1, species_center=3) False
LabelsEntry(order_nu=2, inversion_sigma=-1, spherical_harmonics_l=1, species_center=3) False
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=2, species_center=3) True
LabelsEntry(order_nu=2, inversion_sigma=-1, spherical_harmonics_l=2, species_center=3) True
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=0, species_center=8) True
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=1, species_center=8) False
LabelsEntry(order_nu=2, inversion_sigma=-1, spherical_harmonics_l=1, species_center=8) False
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=2, species_center=8) True
LabelsEntry(order_nu=2, inversion_sigma=-1, spherical_harmonics_l=2, species_center=8) True
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=0, species_center=22) True
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=1, species_center=22) False
LabelsEntry(order_nu=2, inversion_sigma=-1, spherical_harmonics_l=1, species_center=22) False
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=2, species_center=22) True
LabelsEntry(order_nu=2, inversion_sigma=-1, spherical_harmonics_l=2, species_center=22) True

In [ ]:
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=0, species_center=3) False
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=1, species_center=3) True
LabelsEntry(order_nu=2, inversion_sigma=-1, spherical_harmonics_l=1, species_center=3) True
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=2, species_center=3) False
LabelsEntry(order_nu=2, inversion_sigma=-1, spherical_harmonics_l=2, species_center=3) False
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=0, species_center=8) False
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=1, species_center=8) True
LabelsEntry(order_nu=2, inversion_sigma=-1, spherical_harmonics_l=1, species_center=8) True
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=2, species_center=8) False
LabelsEntry(order_nu=2, inversion_sigma=-1, spherical_harmonics_l=2, species_center=8) False
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=0, species_center=22) False
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=1, species_center=22) True
LabelsEntry(order_nu=2, inversion_sigma=-1, spherical_harmonics_l=1, species_center=22) True
LabelsEntry(order_nu=2, inversion_sigma=1, spherical_harmonics_l=2, species_center=22) False
LabelsEntry(order_nu=2, inversion_sigma=-1, spherical_harmonics_l=2, species_center=22) False

In [28]:
lsoap_inv.keys[0]["inversion_sigma"]

1

# Normalization Tests

In [7]:
lsoap_new0 = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers,
    nu_target=2,
    lambdas=lambdas,
    lambda_cut=5,
)
lsoap_new0

TensorMap with 15 blocks
keys: order_nu  inversion_sigma  spherical_harmonics_l  species_center
         2             1                   0                  3
         2             1                   1                  3
                                    ...
         2             1                   2                  22
         2            -1                   2                  22

In [8]:
lsoap_old0 = spherical.lambda_soap_vector(
    frames,
    rascal_hypers,
    lambda_cut=5,
)
# Drop all odd parity keys/blocks
keys_to_drop = Labels(
    names=lsoap_old0.keys.names,
    values=lsoap_old0.keys.values[[v not in lambdas for v in lsoap_old0.keys.column("spherical_harmonics_l")]],
)
lsoap_old0 = equistore.drop_blocks(lsoap_old0, keys=keys_to_drop)
lsoap_old0

TensorMap with 15 blocks
keys: inversion_sigma  spherical_harmonics_l  species_center
             1                   0                  3
             1                   1                  3
                               ...
            -1                   1                  22
            -1                   2                  22

# Test system 1

In [ ]:
%%timeit

use_sparse = False

tensor_dense = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=use_sparse,
)
tensor_dense

# timeit comes out at about 22 seconds for nu_target = 3

In [ ]:
%%timeit

use_sparse = True

tensor_sparse = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=use_sparse,
)
tensor_sparse

# timeit comes out at about 13 seconds for nu_target = 3

In [ ]:
tensor_sparse = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=True,
)

tensor_dense = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
    use_sparse=False,
)

assert equistore.allclose(tensor_dense, tensor_sparse)

# Test system 2

In [ ]:
frames = [ase.io.read("frame.xyz")]

lambdas = np.array([0, 2])

rascal_hypers = {
    "cutoff": 3.0,  # Angstrom
    "max_radial": 6,  # Exclusive
    "max_angular": 5,  # Inclusive
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "center_atom_weight": 1.0,
}

n_body = clebsch_gordan.n_body_iteration_single_center(
    frames,
    rascal_hypers=rascal_hypers,
    nu_target=3,
    lambdas=lambdas,
)
n_body

In [ ]:
# selected_samples = None
# species_neighbors = [1, 8, 6]

# calculator = rascaline.SphericalExpansion(**rascal_hypers)
# nu1 = calculator.compute(frames, selected_samples=selected_samples)

# # Move the "species_neighbor" key to the properties. If species_neighbors is
# # passed as a list of int, sparsity can be created in the properties for
# # these species.
# if species_neighbors is None:
#     keys_to_move = "species_neighbor"
# else:
#     keys_to_move = Labels(
#         names=["species_neighbor"],
#         values=np.array(species_neighbors).reshape(-1, 1),
#     )
# nu1 = nu1.keys_to_properties(keys_to_move=keys_to_move)
# nu1 = clebsch_gordan._add_nu_sigma_to_key_names(nu1)